# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID     City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0  college  64.8569 -147.8028    273.33        95          23   
1        1   namibe -15.1961   12.1522    290.87        80           0   
2        2   pombas  17.1503  -25.0201    297.03        67          59   
3        3   sabang   5.8933   95.3214    300.32        75          81   
4        4   ixtapa  20.7000 -105.2000    301.72        74         100   

   Wind Speed Country        Date  
0        0.97      US  1698714001  
1        1.89      AO  1698714311  
2        1.02      CV  1698714461  
3        1.14      ID  1698714301  
4        2.06      MX  1698714465

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM', frame_width=800, frame_height=500, size='Humidity', color='City')

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_ideal_df = city_data_df.loc[(city_data_df['Max Temp']>=283) & (city_data_df['Max Temp']<=291) & (city_data_df['Humidity']<=75) & \
(city_data_df['Cloudiness']<=15) & (city_data_df['Wind Speed']<4.5),:]

# Drop any rows with null values
# YOUR CODE HERE
city_data_ideal_df.dropna(how='any')

# Display sample data
# YOUR CODE HERE
city_data_ideal_df

City_ID       City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
151      151  tinogasta -28.0632  -67.5649    290.80        52           0   
237      237    fortuna  40.5982 -124.1573    286.85        63           0   
299      299    stawell -37.0500  142.7667    289.06        35           3   
320      320     gijang  35.2442  129.2139    290.00        46           0   
457      457      hotan  37.0997   79.9269    286.04        18           0   
471      471    plastun  44.7561  136.2939    288.07        56           0   
507      507      mutsu  41.2894  141.2169    287.98        68           6   
513      513    baoding  38.8511  115.4903    289.85        49          10   
555      555       faro  37.0194   -7.9322    290.35        72           0   
559      559       argo  37.6333   22.7333    290.25        72           0   
566      566     baiyin  36.5583  104.2081    285.15        35           0   
575      575      surab  28.4914   66.2585    286.32        50           1   

     Wind Speed Country        Date  
151        2.43      AR  1698714643  
237        0.00      US  1698714960  
299        4.26      AU  1698715145  
320        1.49      KR  1698715194  
457        1.71      CN  1698715515  
471        1.68      RU  1698715545  
507        4.24      JP  1698715625  
513        0.93      CN  1698715638  
555        1.54      PT  1698715640  
559        1.34      GR  1698715739  
566        1.70      CN  1698715755  
575        0.79      PK  1698715776

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
city_data_ideal_df.copy()
hotel_df = city_data_ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''

# Display sample data
# YOUR CODE HERE
hotel_df

C:\Users\jerry\AppData\Local\Temp\ipykernel_27820\862735796.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''


City Country      Lat       Lng  Humidity Hotel Name
151  tinogasta      AR -28.0632  -67.5649        52           
237    fortuna      US  40.5982 -124.1573        63           
299    stawell      AU -37.0500  142.7667        35           
320     gijang      KR  35.2442  129.2139        46           
457      hotan      CN  37.0997   79.9269        18           
471    plastun      RU  44.7561  136.2939        56           
507      mutsu      JP  41.2894  141.2169        68           
513    baoding      CN  38.8511  115.4903        49           
555       faro      PT  37.0194   -7.9322        72           
559       argo      GR  37.6333   22.7333        72           
566     baiyin      CN  36.5583  104.2081        35           
575      surab      PK  28.4914   66.2585        50

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [95]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE

categories = 'accommodation.hotel'

params = {'categories': categories, # YOUR CODE HERE
          'api_key': geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"  # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)  # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()  # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tinogasta - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
stawell - nearest hotel: Coorrabin Motor Inn
gijang - nearest hotel: 힐튼호텔 부산
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
plastun - nearest hotel: Россияночка
mutsu - nearest hotel: むつパークホテル
baoding - nearest hotel: Homeinn
faro - nearest hotel: Best Western Hotel Dom Bernardo
argo - nearest hotel: Apollon Hotel
baiyin - nearest hotel: 红鹭宾馆
surab - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
151  tinogasta      AR -28.0632  -67.5649        52   
237    fortuna      US  40.5982 -124.1573        63   
299    stawell      AU -37.0500  142.7667        35   
320     gijang      KR  35.2442  129.2139        46   
457      hotan      CN  37.0997   79.9269        18   
471    plastun      RU  44.7561  136.2939        56   
507      mutsu      JP  41.2894  141.2169        68   
513    baoding      CN  38.8511  115.4903        49   
555       faro      PT  37.0194   -7.9322        72   
559       argo      GR  37.6333   22.7333        72   
566     baiyin      CN  36.5583  104.2081        35   
575      surab      PK  28.4914   66.2585        50   

                                          Hotel Name  
151                                   No hotel found  
237             Comfort Inn & Suites Redwood Country  
299                              Coorrabin Motor Inn  
320                                          힐튼호텔 부산  
457  Hua Yi International Hotel (accepts foreigners)  
471                                      Россияночка  
507                                         むつパークホテル  
513                                          Homeinn  
555                  Best Western Hotel Dom Bernardo  
559                                    Apollon Hotel  
566                                             红鹭宾馆  
575                                   No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [6]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country'],
    geo=True,
    tiles='OSM',
   frame_width=800,
    frame_height=600
)

# Display the map
# YOUR CODE HERE
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)